In [1]:
import pandas as pd
from functools import partial

from IPython.display import display,Markdown

# Load data

Adapters can only be applied in order from lowest to highest jolts,
so we're sorting here, too.

In [2]:
with open('jolts.txt') as fp:
    jolts = sorted([ int(j) for j in fp.read().splitlines() ])

# Question 1

If we're using all the adapters, then the sorted order is the order we 
use them in. Just need to add begining and ending values, compute differences
and count them. This is a one-liner in pandas.

In [3]:
steps = pd.Series([0] + jolts + [jolts[-1] + 3]).diff().value_counts()
steps[1] * steps[3]

1848

# Question 2

After trying something that didn't work, I went back to pandas. The
table I'm creating here has three columns: 

* "jolts" is my list of adapters
* "jump_exists" is a list of next adapter jumps that I can do, given my specific bag of adapters
* "path_count" will be my count of paths from this row of the table to the max jolt value. It's 
  starting life as an empty column here.

I also created a reverse_lookup series for given a jolt value, find the row index in the 
above dataframe, because I'm going to have to do that a lot.

In [4]:
def next_jumps(jolt, jolt_set):
    # legally, can jump 1, 2, or 3 jolts
    legal_jumps = [jolt + i for i in range(1,4)]
    # but maybe not all of them are in my bag of adapters
    jump_exists = [i for i in legal_jumps if i in jolt_set]
    return jump_exists

data = pd.Series([0] + jolts + [jolts[-1] + 3], name='jolts').to_frame()

jolt_set = set(data.jolts)
data['jump_exists'] = data.jolts.apply(partial(next_jumps, jolt_set=jolt_set))

data['path_count'] = pd.NA

# a way to look up the jolt value to the row in the data table
reverse_lookup = pd.Series(index = data.jolts.values, data = data.jolts.index )

# Here's peek at the data table
display(Markdown(data.head(10).to_markdown()))

|    |   jolts | jump_exists   | path_count   |
|---:|--------:|:--------------|:-------------|
|  0 |       0 | [1, 2, 3]     | <NA>         |
|  1 |       1 | [2, 3, 4]     | <NA>         |
|  2 |       2 | [3, 4]        | <NA>         |
|  3 |       3 | [4]           | <NA>         |
|  4 |       4 | [7]           | <NA>         |
|  5 |       7 | [10]          | <NA>         |
|  6 |      10 | [11, 12, 13]  | <NA>         |
|  7 |      11 | [12, 13]      | <NA>         |
|  8 |      12 | [13]          | <NA>         |
|  9 |      13 | [16]          | <NA>         |

We're going to work the problem from the bottom up -- the number of future paths that exist at 
row j are the sum of the counts of future paths for each of the "next jumps" available.

So, we need to seed the bottom of the table with their jump counts. Since we are looking at a maximum
of three legal jumps forward at row j, seeding the bottom three rows will do it. 

Assuming valid input,
* From the last row, there is nowhere to jump, path_count = 0
* From the second to last row, there is only one place to jump, path_count = 1
* From the third to last row, could possibly jump to either or both of those, and we'll know by the length of the legal jump array.

In [5]:
last_index = data.index.max()
data.at[last_index, 'path_count']  = 0 
data.at[last_index - 1, 'path_count']  = 1  
data.at[last_index - 2, 'path_count']  = len(data.at[last_index -2 , 'jump_exists'])  

display(Markdown(data.tail(5).to_markdown()))

|    |   jolts | jump_exists     | path_count   |
|---:|--------:|:----------------|:-------------|
| 90 |     144 | [145, 146, 147] | <NA>         |
| 91 |     145 | [146, 147]      | <NA>         |
| 92 |     146 | [147]           | 1            |
| 93 |     147 | [150]           | 1            |
| 94 |     150 | []              | 0            |

In the above table, row 91 will be filled with the sum of the path_count values from the 
jump_exists values of 146 and 147. Reverse_lookup of 146 takes you
to row 92, and the count is 1. Reverse lookup of 147 takes you to row 93, and that count is also 1.

In [6]:
unfilled_rows = reversed(range(0, last_index-2))
for index in unfilled_rows:
    path_count = 0
    for ex in data.at[index, 'jump_exists']:
        row_num = reverse_lookup[ex]
        path_count += data.at[row_num, 'path_count']
    data.at[index, 'path_count'] = path_count
        
# and another look at the data
display(Markdown(data.tail(20).to_markdown()))

|    |   jolts | jump_exists     |   path_count |
|---:|--------:|:----------------|-------------:|
| 75 |     117 | [118]           |          112 |
| 76 |     118 | [121]           |          112 |
| 77 |     121 | [124]           |          112 |
| 78 |     124 | [125, 126]      |          112 |
| 79 |     125 | [126]           |           56 |
| 80 |     126 | [129]           |           56 |
| 81 |     129 | [132]           |           56 |
| 82 |     132 | [133, 134, 135] |           56 |
| 83 |     133 | [134, 135]      |           28 |
| 84 |     134 | [135]           |           14 |
| 85 |     135 | [138]           |           14 |
| 86 |     138 | [139, 140]      |           14 |
| 87 |     139 | [140]           |            7 |
| 88 |     140 | [143]           |            7 |
| 89 |     143 | [144, 145, 146] |            7 |
| 90 |     144 | [145, 146, 147] |            4 |
| 91 |     145 | [146, 147]      |            2 |
| 92 |     146 | [147]           |            1 |
| 93 |     147 | [150]           |            1 |
| 94 |     150 | []              |            0 |

In [7]:
# and the answer we want is at the top of the table 
display(Markdown(data.head().to_markdown()))
data.at[0, 'path_count']

|    |   jolts | jump_exists   |    path_count |
|---:|--------:|:--------------|--------------:|
|  0 |       0 | [1, 2, 3]     | 8099130339328 |
|  1 |       1 | [2, 3, 4]     | 4628074479616 |
|  2 |       2 | [3, 4]        | 2314037239808 |
|  3 |       3 | [4]           | 1157018619904 |
|  4 |       4 | [7]           | 1157018619904 |

8099130339328